In [1]:
import psycopg2
# import sqlalchemy as sq
import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [2]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [3]:
_ = pd.read_sql("select TOP(1000) * from SampleResults where group_name='Soil Analysis' AND analysis_name LIKE '%Complete Soil Analysis%'",con=conn_lims)

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
_.to_csv("sl.csv")

In [5]:
soil_df = pd.read_csv("input/soil_analysis_cleaned.csv")

In [3]:
len(soil_df)

4038356

In [7]:
soil_df['analysis_name'] = soil_df['analysis_name'].str.split(",")

In [9]:
soil_df = soil_df.explode('analysis_name', ignore_index=True)


In [11]:
soil_df.to_csv("output/soil_single_analysis.csv")

In [2]:
soil_df = pd.read_csv("output/soil_single_analysis.csv")

In [3]:
soil_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),iron,165.00,ppm
1,1,0,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,iron,165.00,ppm
2,2,1,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),cec,16.60,meq/100g
3,3,1,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,cec,16.60,meq/100g
4,4,2,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),aluminium,481.00,ppm
...,...,...,...,...,...,...,...,...
6532520,6532520,4038899,CI118SA1115,2024-04-10 00:00:00.000,Zn,organic_carbon,1.31,%
6532521,6532521,4038899,CI118SA1115,2024-04-10 00:00:00.000,Fe,organic_carbon,1.31,%
6532522,6532522,4038899,CI118SA1115,2024-04-10 00:00:00.000,Mn),organic_carbon,1.31,%
6532523,6532523,4038899,CI118SA1115,2024-04-10 00:00:00.000,Soil sulphate (calcium phosphate),organic_carbon,1.31,%


In [4]:
soil_df = soil_df.dropna(subset="result")

In [5]:
df_ = soil_df[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/soil_unit_per_chemical.csv")

In [6]:
df_ = df_.reset_index()

In [7]:
crop_chem_unit_df = pd.DataFrame()
for crop in df_['analysis_name'].unique():
    print(crop)
    chems_count_ = df_.loc[df_['analysis_name'] == crop]
    print(chems_count_)
    crop_chem_unit_df = pd.DataFrame()
    for crop in df_['analysis_name'].unique():
        print(crop)
        chems_count_ = df_.loc[df_['analysis_name'] == crop]
        duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
        temp_crop_chem_unit_df = pd.DataFrame()
        for index,row in duplicates_chem.iterrows():
            chemical = row['chemical_name']
            chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
            unit = chems_count_2.loc[chems_count_2[0] == max(chems_count_2[0])]['unit_name'].values[0]
            _ = pd.DataFrame({"crop":[crop],"chemical_name":[chemical], "unit_name":[unit]})
            temp_crop_chem_unit_df = pd.concat([temp_crop_chem_unit_df, _])
        crop_chem_unit_df = pd.concat([crop_chem_unit_df, temp_crop_chem_unit_df])
crop_chem_unit_df.to_csv("output/soil_unit_per_chemical_decision.csv")

Complete Soil Analysis with Recommendations
                                    analysis_name chemical_name unit_name  \
0     Complete Soil Analysis with Recommendations          zinc       ppm   
1     Complete Soil Analysis with Recommendations           cec  meq/100g   
2     Complete Soil Analysis with Recommendations       calcium       ppm   
3     Complete Soil Analysis with Recommendations     magnesium       ppm   
4     Complete Soil Analysis with Recommendations     manganese       ppm   
...                                           ...           ...       ...   
5408  Complete Soil Analysis with Recommendations      morgan-k       ppm   
5449  Complete Soil Analysis with Recommendations       calcium    mmol/l   
5451  Complete Soil Analysis with Recommendations         boron    µmol/l   
5697  Complete Soil Analysis with Recommendations        e_coli     cfu/g   
5708  Complete Soil Analysis with Recommendations          gold     mg/kg   

          0  
0     81398  
1  

In [8]:
crop_chem_unit_df.crop.nunique()

95

In [9]:
df = pd.DataFrame()
for index, row in crop_chem_unit_df.iterrows():
    crop = row['crop']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = soil_df.loc[(soil_df['analysis_name'] == crop) & (soil_df['chemical_name'] == chemical) & (soil_df['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [10]:
df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
6168,6168,3457,FA611-161SA0001,2019-04-29 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,1.30,%
6329,6329,3532,FA807-384SA0001,2019-10-30 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,3.37,%
7604,7604,4311,CC004SA0484,2019-05-06 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,1.68,%
8755,8755,5056,CC004SA0485,2019-05-06 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,2.23,%
14040,14040,7262,FA807-384SA0002,2019-10-30 00:00:00.000,Complete Soil Analysis with Recommendations,organic_carbon,2.81,%
...,...,...,...,...,...,...,...,...
1949821,1949821,1291549,CC003SA0055,2015-09-15 00:00:00.000,Saturated Soil Paste Extract,sar,7.89,ppm
1949825,1949825,1291551,CC003SA0054,2015-09-15 00:00:00.000,Saturated Soil Paste Extract,sar,8.58,ppm
5369834,5369834,3385441,CA627SA0002,2023-04-19 12:06:00.000,Salmonella spp,e_coli,600.00,cfu/g
5369848,5369848,3385446,CA627SA0003,2023-04-19 12:06:00.000,Salmonella spp,e_coli,90.00,cfu/g


In [11]:
df.to_csv("output/soil_analysis_cleaned.csv",chunksize=1000)

In [26]:
os.getcwd()

'D:\\Cropnuts\\DSML158\\SoilAnalysis'